In [1]:
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
import random

import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig

In [2]:
training_flow = pd.read_csv('data/training_flow.csv').iloc[:,1:]
training_speed = pd.read_csv('data/training_speed.csv').iloc[:,1:]
training_flow.shape

(69695, 150)

In [3]:
training_flow[training_flow==0]=0.1

In [4]:
training_set_q = torch.from_numpy(training_flow.values).to(torch.float32)

training_set_v = torch.from_numpy(training_speed.values).to(torch.float32)

num_sensor = training_set_q.shape[1]
num_sensor

150

In [5]:
training_set_k = training_set_q/training_set_v
training_set_k.shape

torch.Size([69695, 150])

In [6]:
max_q = training_set_q.max(axis=0)[0]
min_q = training_set_q.min(axis=0)[0]

max_k = training_set_k.max(axis=0)[0]
min_k = training_set_k.min(axis=0)[0]

max_v = training_set_v.max(axis=0)[0]
min_v = training_set_v.max(axis=0)[0]

max_k.shape

torch.Size([150])

In [7]:
# Greenberg
class MFD(Module):
    
    def __init__(self,):
        super(MFD, self).__init__()
        
        self.v_c = Parameter(torch.tensor(min_v))
        self.k_jam = Parameter(torch.tensor(max_k))
        
    def forward(self, k):
        
        v_calibration = self.v_c * torch.log(self.k_jam / k)
        
        return v_calibration

In [8]:
mfd = MFD()

<ipython-input-7-8fe6d3676934>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.v_c = Parameter(torch.tensor(min_v))
<ipython-input-7-8fe6d3676934>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.k_jam = Parameter(torch.tensor(max_k))


In [9]:
# Loss function
mse_loss = nn.MSELoss()
# Optimizers
optimizer = torch.optim.Adam(mfd.parameters(), lr=0.01, betas=(0.5, 0.999))
n_epochs = 20000
batch_size = 512
num_sample = len(training_set_k)

In [10]:
def train(n_epochs):
    t0 = time.time()
    for epoch in range(n_epochs):
        
        optimizer.zero_grad()
        
        idx = random.sample(range(0, num_sample),batch_size)
        
        v_calibration_pred = mfd(training_set_k[idx])
        
        loss_target = Variable(torch.Tensor(v_calibration_pred.shape).fill_(1.0), requires_grad=False)
        loss = mse_loss(v_calibration_pred/training_set_v[idx],loss_target)
        
        loss.backward()
        optimizer.step()
        
        print(
            "[Epoch %d/%d] [loss: %f] [time: %f]"
            % (epoch, n_epochs, loss.item(), time.time()-t0)
        )

In [11]:
train(n_epochs = n_epochs)

[Epoch 0/20000] [loss: 3.684623] [time: 0.028816]
[Epoch 1/20000] [loss: 3.925429] [time: 0.035784]
[Epoch 2/20000] [loss: 3.593470] [time: 0.040762]
[Epoch 3/20000] [loss: 3.339910] [time: 0.044744]
[Epoch 4/20000] [loss: 3.627180] [time: 0.048727]
[Epoch 5/20000] [loss: 3.506827] [time: 0.052709]
[Epoch 6/20000] [loss: 3.691974] [time: 0.055696]
[Epoch 7/20000] [loss: 3.595055] [time: 0.059679]
[Epoch 8/20000] [loss: 3.557987] [time: 0.062665]
[Epoch 9/20000] [loss: 3.790112] [time: 0.066648]
[Epoch 10/20000] [loss: 3.558328] [time: 0.069636]
[Epoch 11/20000] [loss: 3.378503] [time: 0.072623]
[Epoch 12/20000] [loss: 3.815718] [time: 0.075609]
[Epoch 13/20000] [loss: 3.503664] [time: 0.079592]
[Epoch 14/20000] [loss: 3.603989] [time: 0.082578]
[Epoch 15/20000] [loss: 3.570728] [time: 0.085565]
[Epoch 16/20000] [loss: 3.655784] [time: 0.089547]
[Epoch 17/20000] [loss: 3.594722] [time: 0.092534]
[Epoch 18/20000] [loss: 3.634959] [time: 0.095520]
[Epoch 19/20000] [loss: 3.643272] [time: 

[Epoch 162/20000] [loss: 3.350599] [time: 0.585362]
[Epoch 163/20000] [loss: 3.227879] [time: 0.589345]
[Epoch 164/20000] [loss: 3.398432] [time: 0.592331]
[Epoch 165/20000] [loss: 3.314265] [time: 0.596314]
[Epoch 166/20000] [loss: 3.355204] [time: 0.599300]
[Epoch 167/20000] [loss: 3.356740] [time: 0.602288]
[Epoch 168/20000] [loss: 3.359589] [time: 0.606271]
[Epoch 169/20000] [loss: 3.264956] [time: 0.609257]
[Epoch 170/20000] [loss: 3.311757] [time: 0.612244]
[Epoch 171/20000] [loss: 3.334468] [time: 0.615231]
[Epoch 172/20000] [loss: 3.139284] [time: 0.619213]
[Epoch 173/20000] [loss: 3.499163] [time: 0.622200]
[Epoch 174/20000] [loss: 3.468992] [time: 0.626182]
[Epoch 175/20000] [loss: 3.185624] [time: 0.630165]
[Epoch 176/20000] [loss: 3.260986] [time: 0.633152]
[Epoch 177/20000] [loss: 3.532698] [time: 0.637134]
[Epoch 178/20000] [loss: 3.530208] [time: 0.640125]
[Epoch 179/20000] [loss: 3.126817] [time: 0.644104]
[Epoch 180/20000] [loss: 3.322791] [time: 0.647090]
[Epoch 181/2

[Epoch 328/20000] [loss: 2.998979] [time: 1.148880]
[Epoch 329/20000] [loss: 3.136569] [time: 1.152862]
[Epoch 330/20000] [loss: 2.911971] [time: 1.155849]
[Epoch 331/20000] [loss: 2.946769] [time: 1.159831]
[Epoch 332/20000] [loss: 3.023853] [time: 1.162818]
[Epoch 333/20000] [loss: 2.985956] [time: 1.166800]
[Epoch 334/20000] [loss: 3.312297] [time: 1.169787]
[Epoch 335/20000] [loss: 3.100411] [time: 1.173770]
[Epoch 336/20000] [loss: 3.171371] [time: 1.176757]
[Epoch 337/20000] [loss: 3.175063] [time: 1.179744]
[Epoch 338/20000] [loss: 2.931332] [time: 1.182731]
[Epoch 339/20000] [loss: 3.036829] [time: 1.185717]
[Epoch 340/20000] [loss: 3.254571] [time: 1.189699]
[Epoch 341/20000] [loss: 2.808551] [time: 1.192687]
[Epoch 342/20000] [loss: 2.945034] [time: 1.195673]
[Epoch 343/20000] [loss: 3.109142] [time: 1.198660]
[Epoch 344/20000] [loss: 2.898405] [time: 1.201651]
[Epoch 345/20000] [loss: 2.967555] [time: 1.204634]
[Epoch 346/20000] [loss: 3.217024] [time: 1.207620]
[Epoch 347/2

[Epoch 495/20000] [loss: 2.814890] [time: 1.711401]
[Epoch 496/20000] [loss: 2.841607] [time: 1.715384]
[Epoch 497/20000] [loss: 2.885572] [time: 1.718370]
[Epoch 498/20000] [loss: 2.915430] [time: 1.722353]
[Epoch 499/20000] [loss: 2.849296] [time: 1.725340]
[Epoch 500/20000] [loss: 2.857436] [time: 1.729322]
[Epoch 501/20000] [loss: 2.984351] [time: 1.732309]
[Epoch 502/20000] [loss: 2.876190] [time: 1.736291]
[Epoch 503/20000] [loss: 2.796723] [time: 1.739279]
[Epoch 504/20000] [loss: 2.944349] [time: 1.742265]
[Epoch 505/20000] [loss: 2.712533] [time: 1.745252]
[Epoch 506/20000] [loss: 2.910846] [time: 1.749234]
[Epoch 507/20000] [loss: 2.720727] [time: 1.752221]
[Epoch 508/20000] [loss: 2.851068] [time: 1.756203]
[Epoch 509/20000] [loss: 2.720938] [time: 1.759190]
[Epoch 510/20000] [loss: 2.822387] [time: 1.763173]
[Epoch 511/20000] [loss: 2.699197] [time: 1.766160]
[Epoch 512/20000] [loss: 2.856268] [time: 1.770142]
[Epoch 513/20000] [loss: 2.713637] [time: 1.773129]
[Epoch 514/2

[Epoch 660/20000] [loss: 2.564780] [time: 2.271931]
[Epoch 661/20000] [loss: 2.673609] [time: 2.274919]
[Epoch 662/20000] [loss: 2.526911] [time: 2.278901]
[Epoch 663/20000] [loss: 2.520397] [time: 2.281888]
[Epoch 664/20000] [loss: 2.549308] [time: 2.284874]
[Epoch 665/20000] [loss: 2.375535] [time: 2.288857]
[Epoch 666/20000] [loss: 2.489955] [time: 2.291843]
[Epoch 667/20000] [loss: 2.641873] [time: 2.294838]
[Epoch 668/20000] [loss: 2.815513] [time: 2.298813]
[Epoch 669/20000] [loss: 2.728480] [time: 2.301800]
[Epoch 670/20000] [loss: 2.564921] [time: 2.304788]
[Epoch 671/20000] [loss: 2.686095] [time: 2.307773]
[Epoch 672/20000] [loss: 2.568125] [time: 2.311756]
[Epoch 673/20000] [loss: 2.575430] [time: 2.314743]
[Epoch 674/20000] [loss: 2.510750] [time: 2.317730]
[Epoch 675/20000] [loss: 2.545653] [time: 2.321712]
[Epoch 676/20000] [loss: 2.549971] [time: 2.324704]
[Epoch 677/20000] [loss: 2.666708] [time: 2.328681]
[Epoch 678/20000] [loss: 2.530075] [time: 2.331668]
[Epoch 679/2

...

[Epoch 19203/20000] [loss: 0.059216] [time: 64.708836]
[Epoch 19204/20000] [loss: 0.058479] [time: 64.712819]
[Epoch 19205/20000] [loss: 0.058906] [time: 64.715806]
[Epoch 19206/20000] [loss: 0.056923] [time: 64.718792]
[Epoch 19207/20000] [loss: 0.059659] [time: 64.722775]
[Epoch 19208/20000] [loss: 0.058866] [time: 64.725761]
[Epoch 19209/20000] [loss: 0.057955] [time: 64.728748]
[Epoch 19210/20000] [loss: 0.059090] [time: 64.732731]
[Epoch 19211/20000] [loss: 0.060660] [time: 64.735717]
[Epoch 19212/20000] [loss: 0.057001] [time: 64.739700]
[Epoch 19213/20000] [loss: 0.059892] [time: 64.742687]
[Epoch 19214/20000] [loss: 0.056380] [time: 64.745673]
[Epoch 19215/20000] [loss: 0.056457] [time: 64.749656]
[Epoch 19216/20000] [loss: 0.057784] [time: 64.752643]
[Epoch 19217/20000] [loss: 0.059110] [time: 64.755630]
[Epoch 19218/20000] [loss: 0.054996] [time: 64.759612]
[Epoch 19219/20000] [loss: 0.056695] [time: 64.762599]
[Epoch 19220/20000] [loss: 0.057551] [time: 64.766582]
[Epoch 192

[Epoch 19373/20000] [loss: 0.056045] [time: 65.270362]
[Epoch 19374/20000] [loss: 0.056315] [time: 65.274345]
[Epoch 19375/20000] [loss: 0.066210] [time: 65.278327]
[Epoch 19376/20000] [loss: 0.057163] [time: 65.281314]
[Epoch 19377/20000] [loss: 0.058829] [time: 65.285296]
[Epoch 19378/20000] [loss: 0.058425] [time: 65.288283]
[Epoch 19379/20000] [loss: 0.057037] [time: 65.292266]
[Epoch 19380/20000] [loss: 0.063870] [time: 65.295259]
[Epoch 19381/20000] [loss: 0.055419] [time: 65.299235]
[Epoch 19382/20000] [loss: 0.058284] [time: 65.302222]
[Epoch 19383/20000] [loss: 0.058640] [time: 65.305209]
[Epoch 19384/20000] [loss: 0.060013] [time: 65.308196]
[Epoch 19385/20000] [loss: 0.056933] [time: 65.312179]
[Epoch 19386/20000] [loss: 0.059480] [time: 65.315165]
[Epoch 19387/20000] [loss: 0.061068] [time: 65.318152]
[Epoch 19388/20000] [loss: 0.062642] [time: 65.321139]
[Epoch 19389/20000] [loss: 0.057179] [time: 65.324126]
[Epoch 19390/20000] [loss: 0.057825] [time: 65.328109]
[Epoch 193

[Epoch 19541/20000] [loss: 0.062836] [time: 65.834875]
[Epoch 19542/20000] [loss: 0.058160] [time: 65.838858]
[Epoch 19543/20000] [loss: 0.057870] [time: 65.841845]
[Epoch 19544/20000] [loss: 0.057420] [time: 65.844832]
[Epoch 19545/20000] [loss: 0.055906] [time: 65.848813]
[Epoch 19546/20000] [loss: 0.057726] [time: 65.851800]
[Epoch 19547/20000] [loss: 0.057564] [time: 65.854787]
[Epoch 19548/20000] [loss: 0.056389] [time: 65.857774]
[Epoch 19549/20000] [loss: 0.061018] [time: 65.861756]
[Epoch 19550/20000] [loss: 0.057704] [time: 65.864743]
[Epoch 19551/20000] [loss: 0.058195] [time: 65.867730]
[Epoch 19552/20000] [loss: 0.056845] [time: 65.871713]
[Epoch 19553/20000] [loss: 0.059912] [time: 65.874699]
[Epoch 19554/20000] [loss: 0.060159] [time: 65.878682]
[Epoch 19555/20000] [loss: 0.056501] [time: 65.881669]
[Epoch 19556/20000] [loss: 0.058331] [time: 65.884655]
[Epoch 19557/20000] [loss: 0.055274] [time: 65.888638]
[Epoch 19558/20000] [loss: 0.059903] [time: 65.891625]
[Epoch 195

[Epoch 19711/20000] [loss: 0.057365] [time: 66.398392]
[Epoch 19712/20000] [loss: 0.058570] [time: 66.402374]
[Epoch 19713/20000] [loss: 0.057297] [time: 66.405361]
[Epoch 19714/20000] [loss: 0.057450] [time: 66.409343]
[Epoch 19715/20000] [loss: 0.057939] [time: 66.412330]
[Epoch 19716/20000] [loss: 0.056899] [time: 66.416313]
[Epoch 19717/20000] [loss: 0.060793] [time: 66.419300]
[Epoch 19718/20000] [loss: 0.056504] [time: 66.423282]
[Epoch 19719/20000] [loss: 0.060367] [time: 66.426269]
[Epoch 19720/20000] [loss: 0.059089] [time: 66.429256]
[Epoch 19721/20000] [loss: 0.057774] [time: 66.433239]
[Epoch 19722/20000] [loss: 0.057907] [time: 66.436232]
[Epoch 19723/20000] [loss: 0.057283] [time: 66.439213]
[Epoch 19724/20000] [loss: 0.059922] [time: 66.443194]
[Epoch 19725/20000] [loss: 0.059670] [time: 66.446181]
[Epoch 19726/20000] [loss: 0.057404] [time: 66.449168]
[Epoch 19727/20000] [loss: 0.059416] [time: 66.452156]
[Epoch 19728/20000] [loss: 0.057155] [time: 66.455142]
[Epoch 197

[Epoch 19878/20000] [loss: 0.055195] [time: 66.960914]
[Epoch 19879/20000] [loss: 0.059006] [time: 66.964896]
[Epoch 19880/20000] [loss: 0.055781] [time: 66.967883]
[Epoch 19881/20000] [loss: 0.056974] [time: 66.970870]
[Epoch 19882/20000] [loss: 0.059357] [time: 66.973857]
[Epoch 19883/20000] [loss: 0.055315] [time: 66.977845]
[Epoch 19884/20000] [loss: 0.058390] [time: 66.980826]
[Epoch 19885/20000] [loss: 0.061105] [time: 66.984808]
[Epoch 19886/20000] [loss: 0.058769] [time: 66.987795]
[Epoch 19887/20000] [loss: 0.060543] [time: 66.991777]
[Epoch 19888/20000] [loss: 0.057106] [time: 66.994764]
[Epoch 19889/20000] [loss: 0.057907] [time: 66.998747]
[Epoch 19890/20000] [loss: 0.057884] [time: 67.001734]
[Epoch 19891/20000] [loss: 0.057559] [time: 67.004720]
[Epoch 19892/20000] [loss: 0.058350] [time: 67.007708]
[Epoch 19893/20000] [loss: 0.055748] [time: 67.010694]
[Epoch 19894/20000] [loss: 0.058895] [time: 67.014676]
[Epoch 19895/20000] [loss: 0.059331] [time: 67.017663]
[Epoch 198

In [12]:
DataFrame(mfd.v_c.detach().numpy()).to_csv('parameter/GB_v_c.csv',index=False)
DataFrame(mfd.k_jam.detach().numpy()).to_csv('parameter/GB_k_jam.csv',index=False)

In [13]:
# Greenshields
class MFD(Module):
    
    def __init__(self,):
        super(MFD, self).__init__()
        
        self.v_f = Parameter(torch.tensor(max_v))
        self.k_jam = Parameter(torch.tensor(max_k))
        
    def forward(self, k):
        
        v_calibration = self.v_f * (1 - k/self.k_jam)
        
        return v_calibration

In [14]:
mfd = MFD()

<ipython-input-13-b8f41b98dfc7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.v_f = Parameter(torch.tensor(max_v))
<ipython-input-13-b8f41b98dfc7>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.k_jam = Parameter(torch.tensor(max_k))


In [15]:
# Loss function
mse_loss = nn.MSELoss()
# Optimizers
optimizer = torch.optim.Adam(mfd.parameters(), lr=0.01, betas=(0.5, 0.999))
n_epochs = 20000
batch_size = 512
num_sample = len(training_set_k)

In [16]:
def train(n_epochs):
    t0 = time.time()
    for epoch in range(n_epochs):
        
        optimizer.zero_grad()
        
        idx = random.sample(range(0, num_sample),batch_size)
        
        v_calibration_pred = mfd(training_set_k[idx])
        
        loss_target = Variable(torch.Tensor(v_calibration_pred.shape).fill_(1.0), requires_grad=False)
        loss = mse_loss(v_calibration_pred/training_set_v[idx],loss_target)
        
        loss.backward()
        optimizer.step()
        
        print(
            "[Epoch %d/%d] [loss: %f] [time: %f]"
            % (epoch, n_epochs, loss.item(), time.time()-t0)
        )

In [17]:
train(n_epochs = n_epochs)

[Epoch 0/20000] [loss: 0.192682] [time: 0.012938]
[Epoch 1/20000] [loss: 0.169109] [time: 0.022894]
[Epoch 2/20000] [loss: 0.186125] [time: 0.030858]
[Epoch 3/20000] [loss: 0.199504] [time: 0.036832]
[Epoch 4/20000] [loss: 0.183974] [time: 0.040814]
[Epoch 5/20000] [loss: 0.164525] [time: 0.045792]
[Epoch 6/20000] [loss: 0.201953] [time: 0.048779]
[Epoch 7/20000] [loss: 0.182141] [time: 0.052762]
[Epoch 8/20000] [loss: 0.188449] [time: 0.056744]
[Epoch 9/20000] [loss: 0.201131] [time: 0.060726]
[Epoch 10/20000] [loss: 0.171068] [time: 0.064708]
[Epoch 11/20000] [loss: 0.180330] [time: 0.067695]
[Epoch 12/20000] [loss: 0.196213] [time: 0.071678]
[Epoch 13/20000] [loss: 0.197871] [time: 0.075661]
[Epoch 14/20000] [loss: 0.182601] [time: 0.079643]
[Epoch 15/20000] [loss: 0.193157] [time: 0.082629]
[Epoch 16/20000] [loss: 0.196513] [time: 0.086612]
[Epoch 17/20000] [loss: 0.181257] [time: 0.090594]
[Epoch 18/20000] [loss: 0.182124] [time: 0.093582]
[Epoch 19/20000] [loss: 0.209506] [time: 

[Epoch 203/20000] [loss: 0.166208] [time: 0.756660]
[Epoch 204/20000] [loss: 0.170271] [time: 0.760643]
[Epoch 205/20000] [loss: 0.176616] [time: 0.764625]
[Epoch 206/20000] [loss: 0.176220] [time: 0.767612]
[Epoch 207/20000] [loss: 0.162606] [time: 0.771594]
[Epoch 208/20000] [loss: 0.158565] [time: 0.774581]
[Epoch 209/20000] [loss: 0.173451] [time: 0.778563]
[Epoch 210/20000] [loss: 0.170618] [time: 0.781550]
[Epoch 211/20000] [loss: 0.162652] [time: 0.784537]
[Epoch 212/20000] [loss: 0.155027] [time: 0.788525]
[Epoch 213/20000] [loss: 0.157908] [time: 0.791507]
[Epoch 214/20000] [loss: 0.156230] [time: 0.795489]
[Epoch 215/20000] [loss: 0.165588] [time: 0.799471]
[Epoch 216/20000] [loss: 0.157462] [time: 0.802459]
[Epoch 217/20000] [loss: 0.159691] [time: 0.806441]
[Epoch 218/20000] [loss: 0.151640] [time: 0.810423]
[Epoch 219/20000] [loss: 0.156503] [time: 0.813410]
[Epoch 220/20000] [loss: 0.146370] [time: 0.817392]
[Epoch 221/20000] [loss: 0.164229] [time: 0.820379]
[Epoch 222/2

[Epoch 361/20000] [loss: 0.139358] [time: 1.318186]
[Epoch 362/20000] [loss: 0.152697] [time: 1.322168]
[Epoch 363/20000] [loss: 0.156767] [time: 1.326150]
[Epoch 364/20000] [loss: 0.151624] [time: 1.329137]
[Epoch 365/20000] [loss: 0.146944] [time: 1.333120]
[Epoch 366/20000] [loss: 0.150146] [time: 1.337102]
[Epoch 367/20000] [loss: 0.148359] [time: 1.340089]
[Epoch 368/20000] [loss: 0.152393] [time: 1.344071]
[Epoch 369/20000] [loss: 0.154456] [time: 1.347058]
[Epoch 370/20000] [loss: 0.138455] [time: 1.351041]
[Epoch 371/20000] [loss: 0.148643] [time: 1.355023]
[Epoch 372/20000] [loss: 0.153222] [time: 1.358011]
[Epoch 373/20000] [loss: 0.132597] [time: 1.361993]
[Epoch 374/20000] [loss: 0.137047] [time: 1.364980]
[Epoch 375/20000] [loss: 0.154536] [time: 1.368962]
[Epoch 376/20000] [loss: 0.137390] [time: 1.371949]
[Epoch 377/20000] [loss: 0.139796] [time: 1.375932]
[Epoch 378/20000] [loss: 0.148663] [time: 1.378918]
[Epoch 379/20000] [loss: 0.133369] [time: 1.382900]
[Epoch 380/2

[Epoch 521/20000] [loss: 0.134187] [time: 1.881702]
[Epoch 522/20000] [loss: 0.125577] [time: 1.885685]
[Epoch 523/20000] [loss: 0.136525] [time: 1.888672]
[Epoch 524/20000] [loss: 0.153336] [time: 1.892654]
[Epoch 525/20000] [loss: 0.119695] [time: 1.895641]
[Epoch 526/20000] [loss: 0.145313] [time: 1.899623]
[Epoch 527/20000] [loss: 0.145945] [time: 1.903606]
[Epoch 528/20000] [loss: 0.129148] [time: 1.906593]
[Epoch 529/20000] [loss: 0.132371] [time: 1.910575]
[Epoch 530/20000] [loss: 0.141376] [time: 1.913562]
[Epoch 531/20000] [loss: 0.124825] [time: 1.917545]
[Epoch 532/20000] [loss: 0.139276] [time: 1.920532]
[Epoch 533/20000] [loss: 0.129581] [time: 1.924514]
[Epoch 534/20000] [loss: 0.140704] [time: 1.927501]
[Epoch 535/20000] [loss: 0.132083] [time: 1.931483]
[Epoch 536/20000] [loss: 0.138954] [time: 1.934470]
[Epoch 537/20000] [loss: 0.130166] [time: 1.938452]
[Epoch 538/20000] [loss: 0.130413] [time: 1.942435]
[Epoch 539/20000] [loss: 0.126282] [time: 1.945422]
[Epoch 540/2

[Epoch 680/20000] [loss: 0.126932] [time: 2.443228]
[Epoch 681/20000] [loss: 0.125947] [time: 2.447211]
[Epoch 682/20000] [loss: 0.112930] [time: 2.450199]
[Epoch 683/20000] [loss: 0.115361] [time: 2.454181]
[Epoch 684/20000] [loss: 0.114607] [time: 2.457167]
[Epoch 685/20000] [loss: 0.109626] [time: 2.461150]
[Epoch 686/20000] [loss: 0.126695] [time: 2.464137]
[Epoch 687/20000] [loss: 0.117086] [time: 2.468119]
[Epoch 688/20000] [loss: 0.123260] [time: 2.471106]
[Epoch 689/20000] [loss: 0.127567] [time: 2.474093]
[Epoch 690/20000] [loss: 0.115057] [time: 2.477080]
[Epoch 691/20000] [loss: 0.118839] [time: 2.481062]
[Epoch 692/20000] [loss: 0.120500] [time: 2.484049]
[Epoch 693/20000] [loss: 0.116463] [time: 2.487036]
[Epoch 694/20000] [loss: 0.125120] [time: 2.491019]
[Epoch 695/20000] [loss: 0.112469] [time: 2.494006]
[Epoch 696/20000] [loss: 0.113965] [time: 2.497987]
[Epoch 697/20000] [loss: 0.122763] [time: 2.500974]
[Epoch 698/20000] [loss: 0.121525] [time: 2.504956]
[Epoch 699/2

...

[Epoch 19314/20000] [loss: 0.034078] [time: 68.256258]
[Epoch 19315/20000] [loss: 0.029925] [time: 68.260241]
[Epoch 19316/20000] [loss: 0.029739] [time: 68.264223]
[Epoch 19317/20000] [loss: 0.033313] [time: 68.267210]
[Epoch 19318/20000] [loss: 0.031494] [time: 68.271192]
[Epoch 19319/20000] [loss: 0.036498] [time: 68.274179]
[Epoch 19320/20000] [loss: 0.033906] [time: 68.278162]
[Epoch 19321/20000] [loss: 0.030562] [time: 68.282144]
[Epoch 19322/20000] [loss: 0.034005] [time: 68.285131]
[Epoch 19323/20000] [loss: 0.032026] [time: 68.289113]
[Epoch 19324/20000] [loss: 0.032680] [time: 68.292100]
[Epoch 19325/20000] [loss: 0.034679] [time: 68.296082]
[Epoch 19326/20000] [loss: 0.034309] [time: 68.299069]
[Epoch 19327/20000] [loss: 0.029201] [time: 68.302057]
[Epoch 19328/20000] [loss: 0.030168] [time: 68.306039]
[Epoch 19329/20000] [loss: 0.032564] [time: 68.309026]
[Epoch 19330/20000] [loss: 0.031596] [time: 68.312013]
[Epoch 19331/20000] [loss: 0.032070] [time: 68.314999]
[Epoch 193

[Epoch 19474/20000] [loss: 0.030132] [time: 68.818780]
[Epoch 19475/20000] [loss: 0.033897] [time: 68.822762]
[Epoch 19476/20000] [loss: 0.034885] [time: 68.825749]
[Epoch 19477/20000] [loss: 0.035136] [time: 68.828736]
[Epoch 19478/20000] [loss: 0.033224] [time: 68.832719]
[Epoch 19479/20000] [loss: 0.031909] [time: 68.835706]
[Epoch 19480/20000] [loss: 0.031597] [time: 68.838692]
[Epoch 19481/20000] [loss: 0.033202] [time: 68.842674]
[Epoch 19482/20000] [loss: 0.031483] [time: 68.845662]
[Epoch 19483/20000] [loss: 0.032231] [time: 68.848649]
[Epoch 19484/20000] [loss: 0.033375] [time: 68.852631]
[Epoch 19485/20000] [loss: 0.035208] [time: 68.855618]
[Epoch 19486/20000] [loss: 0.032966] [time: 68.859600]
[Epoch 19487/20000] [loss: 0.033859] [time: 68.862587]
[Epoch 19488/20000] [loss: 0.031665] [time: 68.866569]
[Epoch 19489/20000] [loss: 0.030423] [time: 68.869556]
[Epoch 19490/20000] [loss: 0.030623] [time: 68.872544]
[Epoch 19491/20000] [loss: 0.036048] [time: 68.876525]
[Epoch 194

[Epoch 19635/20000] [loss: 0.038613] [time: 69.382297]
[Epoch 19636/20000] [loss: 0.031646] [time: 69.386282]
[Epoch 19637/20000] [loss: 0.031073] [time: 69.389266]
[Epoch 19638/20000] [loss: 0.030258] [time: 69.393249]
[Epoch 19639/20000] [loss: 0.032406] [time: 69.396235]
[Epoch 19640/20000] [loss: 0.035285] [time: 69.399222]
[Epoch 19641/20000] [loss: 0.029969] [time: 69.403205]
[Epoch 19642/20000] [loss: 0.031774] [time: 69.406192]
[Epoch 19643/20000] [loss: 0.032903] [time: 69.410174]
[Epoch 19644/20000] [loss: 0.034228] [time: 69.413161]
[Epoch 19645/20000] [loss: 0.032228] [time: 69.417144]
[Epoch 19646/20000] [loss: 0.034221] [time: 69.420131]
[Epoch 19647/20000] [loss: 0.031200] [time: 69.424113]
[Epoch 19648/20000] [loss: 0.032960] [time: 69.428096]
[Epoch 19649/20000] [loss: 0.031285] [time: 69.432078]
[Epoch 19650/20000] [loss: 0.035170] [time: 69.435064]
[Epoch 19651/20000] [loss: 0.031815] [time: 69.439048]
[Epoch 19652/20000] [loss: 0.038799] [time: 69.442034]
[Epoch 196

[Epoch 19794/20000] [loss: 0.037522] [time: 69.943823]
[Epoch 19795/20000] [loss: 0.032386] [time: 69.947805]
[Epoch 19796/20000] [loss: 0.035483] [time: 69.950793]
[Epoch 19797/20000] [loss: 0.032411] [time: 69.953779]
[Epoch 19798/20000] [loss: 0.033335] [time: 69.957762]
[Epoch 19799/20000] [loss: 0.032844] [time: 69.960748]
[Epoch 19800/20000] [loss: 0.030517] [time: 69.963736]
[Epoch 19801/20000] [loss: 0.033083] [time: 69.967718]
[Epoch 19802/20000] [loss: 0.031722] [time: 69.971700]
[Epoch 19803/20000] [loss: 0.033673] [time: 69.974687]
[Epoch 19804/20000] [loss: 0.032363] [time: 69.978669]
[Epoch 19805/20000] [loss: 0.032879] [time: 69.981656]
[Epoch 19806/20000] [loss: 0.033629] [time: 69.985639]
[Epoch 19807/20000] [loss: 0.034937] [time: 69.989621]
[Epoch 19808/20000] [loss: 0.031880] [time: 69.992608]
[Epoch 19809/20000] [loss: 0.035664] [time: 69.996590]
[Epoch 19810/20000] [loss: 0.034462] [time: 70.000573]
[Epoch 19811/20000] [loss: 0.035872] [time: 70.003560]
[Epoch 198

[Epoch 19953/20000] [loss: 0.033536] [time: 70.506344]
[Epoch 19954/20000] [loss: 0.031776] [time: 70.510327]
[Epoch 19955/20000] [loss: 0.031932] [time: 70.514309]
[Epoch 19956/20000] [loss: 0.035837] [time: 70.517296]
[Epoch 19957/20000] [loss: 0.033674] [time: 70.521279]
[Epoch 19958/20000] [loss: 0.034034] [time: 70.525261]
[Epoch 19959/20000] [loss: 0.034330] [time: 70.528248]
[Epoch 19960/20000] [loss: 0.034003] [time: 70.532230]
[Epoch 19961/20000] [loss: 0.035840] [time: 70.535218]
[Epoch 19962/20000] [loss: 0.032940] [time: 70.539201]
[Epoch 19963/20000] [loss: 0.033227] [time: 70.543182]
[Epoch 19964/20000] [loss: 0.034241] [time: 70.546169]
[Epoch 19965/20000] [loss: 0.031002] [time: 70.549156]
[Epoch 19966/20000] [loss: 0.032511] [time: 70.553138]
[Epoch 19967/20000] [loss: 0.030520] [time: 70.556125]
[Epoch 19968/20000] [loss: 0.032511] [time: 70.560107]
[Epoch 19969/20000] [loss: 0.031858] [time: 70.563094]
[Epoch 19970/20000] [loss: 0.030047] [time: 70.567077]
[Epoch 199

In [18]:
DataFrame(mfd.v_f.detach().numpy()).to_csv('parameter/GS_v_f.csv',index=False)
DataFrame(mfd.k_jam.detach().numpy()).to_csv('parameter/GS_k_jam.csv',index=False)